# How to fill the input file 

In the folder inputs you will find the document named **input_template.xlms** which contains:
    - the columns read by the pipeline
    - comments below each column name that describe what information should be filled in the column
    - example entry information (what has to be change depending on the analysis)
    
The structure of the columns is exempified in the screenshot of the first column of the file: 
    
<img src="./images/column1.png" alt="column 1" title="Entries of the first column" width="250" height="100" />
    
<!--![structure of columns](column1.png "Entries of the first column")-->
*The first two line contains respectively the column identifier and the description of its content. The lines containing ch4, nh4 and lac__L are the entries used in this example. The entries are written as BiGG identifiers without adding the compartment (e.g. _e or _p or _e)*  

<p style="color: red;"> Compiling .xmls files is easier, but <strong>the pipeline reads .csv files </strong>, therefore once you've finished filling the information in the input_template.xmls file it is suggested to <strong>save the file in CSV format </strong> (with .csv extension) giving it a name that suits your analysis. </p>

###### The comments give specific indication of how to fill the document. Below there is a summerised description of the content.

**Reference models**
The model of the organism that has to be engineered should have metabolites, compartments and reactions in the BiGG namespaces. For this reason, the easiest is to download the model directly from the BiGG database. The input file should contain the ID of the reference organism (*reference_model_ID*) as well as the universal model (*universal_reaction_DB*) that the pipeline should consider as the reactions database. Accepted entries for the latter are bacteria, grampos (for Gram-positive bacteria specific model), gramneg (for Gram-negative bacteria specific model), cyanobacteria, archaea. These entries are used by the pipeline to read one of the 5 universal model variants obtained from the Gitalb repository of CarveMe tool [1] and hosted in the pipeline's directory called inputs.


**Metabolites**
In the input file one carbon source and one product must be indicated writing their BiGG identifier in the first column (*metabolite_BiGG_ID*). One compound should always be indicated as carbon source (*carbon_source*). A negative value can be used to specify the maximum uptake rate of the carbon source during production of the target (*consumption*). If not indicated -100 will be set by default. If there is no interest in finding a strain optimally producing a target product and intermediate of the degradation/assimilation pathway has to be selected anyway. If other compounds of interest are indicated in the first column, the fluxes through their exchange reaction will be checked throughout the pipeline, but their consumption or production will not be optimized. 

**Carbon sourse in the natural environment**
The users have to indicate which compound is used as preferred carbon source in the natural environment (*preferred_c_source*). It usually corresponds to glucose. This information is necessary to calculate the growth rate in the “wild type” model and thus constraint the upper bound of the biomass synthesis reaction. A negative value indicating the uptake rate of the preferred substrate should be indicated (*max_average_uptake*).

**Reactions**
Reactions that the user knows will be needed for the uptake or production of the compounds can be added in the columns G-O. Several information is needed: Reaction ID, name, equation, bounds and Enzyme Commission number are the minimal required information.  

It is suggested to always include transport reactions for all metabolites to assure their correct location in the model compartments. Exchange reactions and transport reactions are default added to the reference model if they are not already present. During the step of universal model expansion, the other reactions are added to it if not present yet. 

The reaction equations should be written (*Reaction_equation*) with all metabolites in BiGG namespace including the compartment indications ( *_e, _p, _c*). Allowed arrows are -->, <--, <=>. These reaction equations will be read as strings and the build_reaction_from_string function of COBRApy [2] package is used by the pipeline to create the *cobra.Reaction* objects and add them to the models. 

**<p style="color: red;"> Tips about looking for reactions in BiGG</p>**

Since the reactions have to be written in BiGG namespace, the advised procedure before adding reactions is to:
1. look for the reaction name in BiGG and see if the reaction of interest if found. 
Typing "methanol dehydrogenase" in the search bar the following results are obtained:

<img src="./images/ALCD1.png" alt="First result for the search of methanol dehydrogenase" title="First result for the search of methanol dehydrogenase" width="350" />

2. check if the reaction is already present in the model of interest 
Typing on ALCD1 you get redrected on this page, where you can see what the reaction looks like (its equation) and in which models it can be found

<img src="./images/ALCD1_models.png" alt="Models containing ALCD1" title="Models containing ALCD1" width="350" />

In this case ALCD1 is not present in the model, but it is present BiGG database, hence it is already in the universal model and does not need to be added. If added, the pipeline will recognize that it is already in the reaction database and avoid adding it again. 

If you cannot find a reaction in BiGG that's where you should write in down in the input file.

**Information needed for the thermodynamic analysis**
One of the information used for the pFBA and MDF module is the indication on the metabolism type (*aerobic*): if the conversion is known to happen in oxic conditions it should be indicated. This information is used to prepare the model for pFBA. The numbers indicating the minimal and maximal concentrations of the substrate (*c_source_concentration_range*) in mM are also needed for MDF calculation. The physiological default one are 0.001 and 10 mM.	

**Scored criteria**
At least 5 criteria (*criteria*) are used to compare the solutions found by the pipeline relatively to the reaction addition and the uptake and production fluxes. 

<img src="./images/criteria.png" alt="column 1" title="Criteria" width="250" height="100" />

Three of the entries in this column should be kept unchanged: 
- intervention
- mdf and 
- path_length. 

Criteria on uptake rate and production rate should be written with the following syntax: *consumption_* and *production_* followed by the identifier of substrate and target compounds respectively. The identifiers should be written without the compartment letter. To idicate the importance of each criteria positive integers should be written in column S (*weight*). The higher the number, the higher the weight for the criteria will be. The number in column S are automatically normed in column T (*normed_weight*). The normed weights are obtained by dividing the values from column S by their sum (U3) resulting in numbers between 0 and 1. Those are the weight used for each criteria. This calculations are done automatically with formulas set in the .xmls file, however when the file is saved as .csv only the vlaues are maintaned, not the forumulas. Therefore, changes to the weights should be done in the .xmls file.

**Information needed for the output files**
The balanced equation of the reaction should be written in the input file (*balanced_equation*). It helps defining the aim of the design and is included in the summary_output.csv file. However, this information is not used by the pipeline.Additioanally, a prerequisite of the thermodynamic module is the knowledge of the stoichiometric coefficients in the conversion of the substrate into the product. The user has indeed has to indicate the coefficients in the input file (*coefficient_c_source* and *coefficient_product*). The one of the of the substrate should be negative. The one of the target product should be positive. 

**Information needed for codon harmonizing the Biobrick sequences matching the added reaction**  
For the codon harmonisation, the compressed .fna.gz file of the multi-sequence CDS of the reference organism (expression host) should be indicated (*path_to_fna_file_expression_host*). This is used for building the codon frequency table. The file should be located in pipeline/inputs directory. This file should be downloaded from [NCBI Assembly database](https://www.ncbi.nlm.nih.gov/assembly).

**Information needed for Gapfilling and OptKnock analysis**
The input file should include the indication of the number of iterations of the Gapfilling algorithm (*iterations*). Finally, the user should indicate (*Optknock*) if the pipeline should also consider reactions knock-outs in the design strategies. If so, Optknock algorithm is used. 


1. D. Machado, S. Andrejev, M. Tramontano, and K. R. Patil, “Fast automated reconstruction of genome-scale metabolic models for microbial species and communities,” Nucleic Acids Res., vol. 46, no. 15, pp. 7542–7553, 2018, doi: 10.1093/nar/gky537.

2. A. Ebrahim, J. A. Lerman, B. O. Palsson, and D. R. Hyduke, “COBRApy: COnstraints-Based Reconstruction and Analysis for Python,” BMC Syst. Biol., vol. 7, no. 1, p. 74, Aug. 2013, doi: 10.1186/1752-0509-7-74.